# Summarization via BART model (pretrained on the CNNDM) and simplification via T5 model (pretrained on the wikilarge)

## Problems:  
1. factual errors after summarization (the reason could be in the small amount of sentenses)
2. 

## TODO:  
1. try different amount of sentences for summarization  
2. try another models (maybe smaller models will work better)

### Summarization

#### BRIO

In [1]:
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration
import pandas as pd
# import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize

IS_CNNDM = True # whether to use CNNDM dataset or XSum dataset
LOWER = False

res = {"document": [], "summary_brio": [], "summary_bart": [], "summary_t5": []}
# Load our model checkpoints
if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
    tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
else:
    model = PegasusForConditionalGeneration.from_pretrained('Yale-LILY/brio-xsum-cased')
    tokenizer = PegasusTokenizer.from_pretrained('Yale-LILY/brio-xsum-cased')

df = pd.read_csv('resources/datasets/epfl_news/epfl_news_filtered_data_train.csv')
# number of sentences
n = 4
# number of examples in exp
for num in range(5):
    summary_full = []
    doc = sent_tokenize(df.document.iloc[num])
    sents = [' '.join(doc[i:i+n]) for i in range(0, len(doc), n)]
    if len(doc) % n != 0:
        sents[-2:] = [' '.join(sents[-2:])]
    for ARTICLE_TO_SUMMARIZE in sents:
        max_length = 512
        # generation example
        if LOWER:
            article = ARTICLE_TO_SUMMARIZE.lower()
        else:
            article = ARTICLE_TO_SUMMARIZE
        inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
        # Generate Summary
        summary_ids = model.generate(inputs["input_ids"])
        summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        summary_full.append(summary)
        # print('Original: ', ARTICLE_TO_SUMMARIZE)
        # print('Summary: ', summary)
        # print()
    res["document"].append(' '.join(sents))
    res["summary_brio"].append(' '.join(summary_full))

/home/blinova/.conda/envs/simple/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### BART

In [2]:
from transformers import pipeline
import pandas as pd
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

df = pd.read_csv('resources/datasets/epfl_news/epfl_news_filtered_data_train.csv')
n = 4
for num in range(5):
    summary_full = []
    doc = sent_tokenize(df.document.iloc[num])
    sents = [' '.join(doc[i:i+n]) for i in range(0, len(doc), n)]
    if len(doc) % n != 0:
        sents[-2:] = [' '.join(sents[-2:])]
    for ARTICLE in sents:
        # print('Original: ', ARTICLE)
        # print('Summary: ', summarizer(ARTICLE, min_length=30, do_sample=False)[0]['summary_text'])
        # print()
        # res["summary_bart"].append(summarizer(ARTICLE, min_length=30, do_sample=False)[0]['summary_text'])
        summary_full.append(summarizer(ARTICLE, min_length=30, do_sample=False)[0]['summary_text'])
    res["summary_bart"].append(' '.join(summary_full))

Your max_length is set to 142, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)
Your max_length is set to 142, but you input_length is only 90. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 142, but you input_length is only 119. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 142, but you input_length is only 117. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)
Your max_length is set to 142, but you input_length is only 129. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)
Your max_length is set to 142, but you input_length is only 116. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)
Your max_length is set to 142, but you input_length is only 84. You might cons

#### T5

In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
df = pd.read_csv('resources/datasets/epfl_news/epfl_news_filtered_data_train.csv')
n = 4
for num in range(5):
    summary_full = []
    doc = sent_tokenize(df.document.iloc[num])
    sents = [' '.join(doc[i:i+n]) for i in range(0, len(doc), n)]
    if len(doc) % n != 0:
        sents[-2:] = ['\n'.join(sents[-2:])]
    for ARTICLE in sents:
        t5_prepared_Text = "summarize: "+ARTICLE
        # print ("original text preprocessed: \n", ARTICLE)
        tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


        # summmarize 
        summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_full.append(output)
        # print ("\n\nSummarized text: \n",output)
    res["summary_t5"].append(' '.join(summary_full))
result_cnndm = pd.DataFrame(res)
result_cnndm.to_csv('resources/datasets/epfl_news/summary_4sent.csv')

In [1]:
# from IPython.display import display, HTML
# raw_html = pd.DataFrame.from_dict(result_cnndm).to_html()
# display(HTML(raw_html))

### Simplification

In [4]:
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
import pandas as pd
from Ts_T5 import T5FineTuner

print("******** LOAD MODEL ********")
model = T5FineTuner.load_from_checkpoint('experiments/exp_wikilarge_oldloss/checkpoint-epoch=5.ckpt')
print('******** GENERATE SIMPLE DOCUMENT ********')
df = pd.read_csv("resources/datasets/epfl_news/summary_4sent.csv")

def generate_simple_document(doc):
    split_lst = sent_tokenize(doc)
    gen_list = []
    for i, seq in enumerate(split_lst):
        gen = model.generate(seq)
        gen_list.append(gen)
    return " ".join(gen_list)

df['simple_brio'] = df['summary_brio'].apply(lambda x:generate_simple_document(x))
df['simple_bart'] = df['summary_bart'].apply(lambda x:generate_simple_document(x))
df['simple_t5'] = df['summary_t5'].apply(lambda x:generate_simple_document(x))
df['simple_document'] = df['document'].apply(lambda x:generate_simple_document(x))
df.to_csv('resources/datasets/epfl_news/sum_simple_4sent.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/blinova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/blinova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


******** LOAD MODEL ********
******** GENERATE SIMPLE DOCUMENT ********


In [7]:
df.to_excel('resources/datasets/epfl_news/sum_simple_4sent.xlsx') 

### Length analysis

In [43]:
print('Compare lens for BRIO')
print('Len of the complex document', len(df.document.iloc[0].split(' ')))
print('Len of the simple document', len(df.simple_document.iloc[0].split(' ')))
print('Len of the summary document', len(df.summary_brio.iloc[0].split(' ')))
print('Len of the simle summary document', len(df.simple_brio.iloc[0].split(' ')))

Len of the complex document 649
Len of the simple document 628
Len of the summary document 285
Len of the simle summary document 293


In [5]:
from IPython.display import display, HTML
raw_html = pd.DataFrame.from_dict(df).to_html()
display(HTML(raw_html))

,Unnamed: 0,document,summary_brio,summary_bart,summary_t5,simple_brio,simple_bart,simple_t5,simple_document
0,0,"The rising number of drones in air space poses numerous challenges. Topping that list is our ability to simply detect these small unmanned aerial vehicles. Periodic near-misses between drones and large airplanes raise the specter of disaster, and the drones themselves often lack the necessary technology to locate other moving objects. To address these issues, EPFL researchers have developed algorithms capable of detecting and tracking small flying objects using a simple camera.\nThe proof of concept was conducted as part of a PhD dissertation, and a real-time detection and collision avoidance system is now being developed in a project funded by the Commission for Technology and Innovation (CTI).Today’s collision avoidance systems operate actively: an airplane in flight calculates its position, altitude and course, and communicates this information to other aircraft using the same technology. Those aircraft can then evaluate the risk of a collision based on their own positioning data and, if necessary, alert the pilot. But this system is only effective as long as all aircraft are equipped with the same technology. In reality, drones often lack such systems, which are costly and heavy and consume more power.Artificial intelligence and Deep learningA camera can thus be an effective, non-cooperative (i.e., not every aircraft must be equipped with it) addition to that system, provided the camera can successfully detect a flying drone.\nTherein lay the obstacle that researchers at EPFL's Computer Vision Laboratory (CVLAB) sought to overcome. The biggest challenge for a moving camera is to spot another moving object. This is much more difficult on a drone than it is on a car, which only moves in two dimensions. Drones move in three dimensions, and the camera is called on to detect objects against the sky or the ground, depending on the angle of sight.\nPlus, drones need to locate objects as quickly as possible, such as when they are still fuzzy black dots against a dark forest. And the fact that no two drones look alike anymore – new models are constantly being developed – meant that the researchers had to find a way to teach the camera to recognize all sorts of drones.In his thesis, Artem Rozantsev showed that these challenges can be overcome. The first step consisted in using artificial intelligence and deep learning to teach the camera to recognize drones. His method combined information on both appearance (types of drones, position, etc.)\nand motion (movement in the camera's field of view), as neither alone was capable of achieving sufficiently reliable detection. He therefore proposed a machine-learning technique that operates on spatio-temporal cubes of image intensities where individual patches are aligned using a regression-based motion stabilization algorithm.Real-time performance and accuracyBut the recognition algorithm on its own was not enough. To train a detector to recognize all types of drones in all kinds of positions, it has to have ""seen"" as many as possible. The existing database of images, however, is limited.\nSo Rozantsev filled in the gaps by generating realistic synthetic images. The generated images, which are based only on a small set of real examples and a coarse 3D model of the object, are used together with the real examples to train the detector. A key ingredient to his method, the generated images are as close as possible to the real ones – not in terms of image quality, but according to the features used by the machine-learning algorithm.The researchers managed to develop a reliable algorithm capable of detecting a drone using a lightweight camera similar to those found in smartphones. The aim of the project, now financed by the CTI, is to train a detector using an even larger data set to improve its real-time performance and accuracy. EPFL's CVLAB researchers are working on this in coll